In [2]:
import os

import pandas as pd

""" 
2/13/23 - All methods related to decoding substrokes, wuickly made as extentions of dPCA, but not
actually using dpca at all.

Decodes both shape and motor variables, various methods of conjunction.

Goal: testing whether shape is strongly different for PMv.
Whereas for M1 the opposite is true (strong motor, conditioned on shape).

Script: analyquick_decode_substrokes.py 

"""

' \n2/13/23 - All methods related to decoding substrokes, wuickly made as extentions of dPCA, but not\nactually using dpca at all.\n\nDecodes both shape and motor variables, various methods of conjunction.\n\nGoal: testing whether shape is strongly different for PMv.\nWhereas for M1 the opposite is true (strong motor, conditioned on shape).\n\nScript: analyquick_decode_substrokes.py \n\n'

In [6]:
"""
Quickly test decoding of both shape and motor-related params for each substroke.
This is all very scratch.

Notebook: 240128_snippets_demixed_PCA (althought this has nothing to do with that code).

"""

import numpy as np
from pythonlib.tools.plottools import savefig
from pythonlib.globals import PATH_ANALYSIS_OUTCOMES
import os
import sys
import pandas as pd
from pythonlib.tools.expttools import writeDictToTxt

DEBUG = False

from neuralmonkey.scripts.analyquick_decode_substrokes import plot_summary_decode_results, load_saved_data, check_var_conti_discr


In [10]:
"""
Quickly test decoding of both shape and motor-related params for each substroke.
This is all very scratch.

Notebook: 240128_snippets_demixed_PCA (althought this has nothing to do with that code).

"""

import numpy as np
from pythonlib.tools.plottools import savefig
from pythonlib.globals import PATH_ANALYSIS_OUTCOMES
import os
import sys
import pandas as pd
from pythonlib.tools.expttools import writeDictToTxt
import matplotlib.pyplot as plt

DEBUG = True

############### PARAMS
animal = "Pancho"
date = 220719
exclude_bad_areas = True
SPIKES_VERSION = "tdt" # since Snippets not yet extracted for ks
bin_by_time_dur = 0.05
bin_by_time_slide = 0.025
which_level = "substroke"

# METHOD 1 - Standard, running separately for each PA
question = "SS_shape"
slice_agg_slices = None
slice_agg_vars_to_split = None
slice_agg_concat_dim = None

# list_time_windows = [(-0.3, 0.)]
# list_time_windows = [(-0.3, 0), (0., 0.3)]
list_time_windows = [(-0.3, 0)]
events_keep = ["00_substrk"]
# list_time_windows = [(-0.3, 0.)]
# events_keep = ["06_on_strokeidx_0"]
print(list_time_windows)

ev = "00_substrk"
thresh_frac_var = 0.85 # PCA, cumvar
n_min_across_all_levs_var = 5

# Demixed PCA
list_dpca_marginalization = ["shape", None]

########################################## RUN
# Load q_params
from neuralmonkey.analyses.rsa import rsagood_questions_dict, rsagood_questions_params
q_params = rsagood_questions_dict(animal, date, question)[question]
# Load data
from neuralmonkey.classes.population_mult import dfallpa_extraction_load_wrapper

combine_into_larger_areas = True
HACK_RENAME_SHAPES = False
if DEBUG:
    substrokes_plot_preprocess = False
else:
    substrokes_plot_preprocess = True
DFallpa = dfallpa_extraction_load_wrapper(animal, date, question, list_time_windows,
                                          which_level=which_level,
                                        bin_by_time_dur = bin_by_time_dur,
                                        bin_by_time_slide = bin_by_time_slide,
                                          combine_into_larger_areas = combine_into_larger_areas,
                                          exclude_bad_areas = exclude_bad_areas,
                                          SPIKES_VERSION = SPIKES_VERSION,
                                          HACK_RENAME_SHAPES = HACK_RENAME_SHAPES, 
                                          substrokes_plot_preprocess=substrokes_plot_preprocess)



SAVEDIR_ANALYSES = f"{PATH_ANALYSIS_OUTCOMES}/recordings/main/DECODE"

list_br = sorted(DFallpa["bregion"].unique().tolist())
if DEBUG:
    list_br = ["M1"]


for prune_to_first_substroke in [True, False]:


    if True:
        # Continuous
        list_var_decode = ["velocity", "angle", "distcum", "circ_signed"]
        list_vars_others = [["shape"], ["shape"], ["shape"], ["shape"]]
        # list_vars_others = [["dummy"], ["dummy"], ["dummy"], ["dummy"]]
    else:
        # Discrete
        list_var_decode = ["shape", "di_an_ci_ve_bin"]
        list_vars_others = [["di_an_ci_ve_bin"], ["shape"]]

    # Combine all
    if DEBUG:
        list_var_decode = ["shape"] + ["shape"] + ["velocity"]
        list_vars_others = [["ss_this_ctxt"], ["di_an_ci_ve_bin"]] + [["shape"]]
    else:
        # list_var_decode = ["shape", "di_an_ci_ve_bin"] + ["velocity", "angle", "distcum", "circ_signed"]
        # list_vars_others = [["di_an_ci_ve_bin"], ["shape"]] + [["shape"], ["shape"], ["shape"], ["shape"]]
        list_var_decode = ["shape", "shape", "di_an_ci_ve_bin"] + ["angle"]
        list_vars_others = [["ss_this_ctxt"], ["di_an_ci_ve_bin"], ["shape"]] + [["shape"]]

    if prune_to_first_substroke==False:
        # Then append index within stroke
        tmp =[]
        for x in list_vars_others:
            tmp.append(list(x) + ["index_within_stroke"])
        list_vars_others = tmp

    from pythonlib.tools.listtools import tabulate_list
    # Count the lowest n data across classes.
    from neuralmonkey.analyses.decode_good import decode_categorical_wrapper

    if DEBUG:
        list_time_windows = [list_time_windows[0]]

    for twind in list_time_windows:

        for dpca_marginalization in list_dpca_marginalization:
            # a = stringify_list(list_time_windows, return_as_str=True)
            # b = stringify_list(events_keep, return_as_str=True)
            SAVEDIR = f"{SAVEDIR_ANALYSES}/{animal}-{date}/{question}/firstsubstrk_{prune_to_first_substroke}-twind_{twind}-dpca_marg_{dpca_marginalization}"
            os.makedirs(SAVEDIR, exist_ok=True)

            params_this = {
                "n_min_across_all_levs_var":n_min_across_all_levs_var,
                "thresh_frac_var":thresh_frac_var,
                "prune_to_first_substroke":prune_to_first_substroke,
            }
            writeDictToTxt(params_this, f"{SAVEDIR}/params.txt")

            ##### Condition on one variable and test decoding (within that variable)

            RES_SINGLE = [] # Single decoder across all data (after pruning to have conjunctions)
            RES = [] # Separate decoder for each lev of conj
            for br in list_br:
                # br = "PMv"

                savedir_preprocess = f"{SAVEDIR}/preprocess_plots/{br}"
                os.makedirs(savedir_preprocess, exist_ok=True)

                from neuralmonkey.analyses.state_space_good import popanal_preprocess_scalar_normalization
                from pythonlib.tools.pandastools import append_col_with_grp_index
                from neuralmonkey.analyses.rsa import preprocess_rsa_prepare_popanal_wrapper
                from neuralmonkey.scripts.analy_dpca_script_quick import plothelper_get_variables

                # Extract PA
                tmp = DFallpa[(DFallpa["bregion"]==br) & (DFallpa["event"]==ev) & (DFallpa["twind"]==twind)]
                assert len(tmp)==1
                pa = tmp["pa"].values[0]

                # Restrict analysis to just first substroke
                if prune_to_first_substroke:
                    pa = pa.slice_by_labels("trials", "index_within_stroke", [0])
                    
                
                ###### USE PCS OR DEMIXED PCS
                if dpca_marginalization is not None:
                    
                    ##### Use dPCA projections for decoding
                    from neuralmonkey.scripts.analy_dpca_script_substrokes import dpca_compute_pa_to_space, transform_from_pa, transform_trial

                    dpca, Z, R, trialR, map_var_to_lev, map_grp_to_idx, params_dpca, panorm = dpca_compute_pa_to_space(
                        pa, [dpca_marginalization], keep_all_margs=False)
                    if dpca_marginalization == "shape":
                        marginalization = "s"
                    elif dpca_marginalization == "dist_angle":
                        marginalization = "m" # motor
                    else:
                        print(dpca_marginalization)
                        assert False

                    # First, convert to final data using PA (e.g., scalar)
                    panorm_scal = panorm.agg_wrapper("times")
                    # dflab = panorm_scal.Xlabels["trials"]
                    trialX_proj = transform_from_pa(dpca, panorm_scal, marginalization) # (ndims, ntrials, 1)

                    # Prune to dims with >0.1% explained variance, or 3, whichever larger
                    tmp = np.argwhere(np.array(dpca.explained_variance_ratio_[marginalization])>0.001)
                    if len(tmp)==0:
                        ndims = 4
                    else:
                        ndims = max(tmp)+1
                        ndims = max([4, ndims]) # take at least 3 dimensions
                        ndims = int(ndims)
                    trialX_proj = trialX_proj[:ndims, :, :]

                    fig, ax = plt.subplots()
                    ax.plot(dpca.explained_variance_ratio_[marginalization])
                    ax.axvline(ndims, color="r")
                    ax.set_title("var exp, dpca, vline=ndims taken")
                    savefig(fig, f"{savedir_preprocess}/demixed_pca_explainedvar.pdf")

                    assert trialX_proj.shape[2]==1
                    trialX_proj = trialX_proj.squeeze()
                    X_orig = trialX_proj.T # (ntrials, nchans)

                else:
                    # PCA.
                    # Standard preprocessing
                    if False:
                        # Dont do this; will do later in context of decoding.
                        pa, res_check_tasksets, res_check_effectvars = preprocess_rsa_prepare_popanal_wrapper(pa, **q_params)
                    _, panorm_scal, _, _, _, _ = popanal_preprocess_scalar_normalization(pa, None, DO_AGG_TRIALS=False)

                    # PCA
                    trialX = panorm_scal.X.copy()
                    assert trialX.shape[2]==1
                    trialX = trialX.squeeze()
                    X_orig = trialX.T # (ntrials, nchans)

                    # Prune with PCA
                    from sklearn.decomposition import PCA
                    pca = PCA(n_components=None)
                    Xpca = pca.fit_transform(X_orig) # (ntrials, ndims)
                    cumvar = np.cumsum(pca.explained_variance_ratio_)
                    npcs_keep = np.argmin(np.abs(cumvar - thresh_frac_var))

                    fig, ax = plt.subplots()
                    ax.plot(pca.explained_variance_ratio_)
                    ax.axvline(npcs_keep, color="r")
                    X_orig = Xpca[:, :npcs_keep]
                    savefig(fig, f"{savedir_preprocess}/pca_explainedvar.pdf")


                # Get labels
                dflab = panorm_scal.Xlabels["trials"].copy()

                for var_decode, vars_others in zip(
                        list_var_decode,
                        list_vars_others):

                    map_levother_to_labels = {}

                    from pythonlib.tools.pandastools import extract_with_levels_of_conjunction_vars
                    # var_decode = "shape"
                    # vars_others = ["di_an_ci_ve_bin"]

                    conti_or_discr = check_var_conti_discr(var_decode)

                    if conti_or_discr=="discr":
                        # var_decode = "di_an_ci_ve_bin"
                        # vars_others = ["shape"]

                        Labels_orig = panorm_scal.Xlabels["trials"][var_decode]
                        assert len(Labels_orig)==X_orig.shape[0]
                        Labels_orig_int, map_int_to_lab = Labels_orig.factorize()
                        map_int_to_lab = {i:lab for i, lab in enumerate(map_int_to_lab)}

                        writeDictToTxt({"map_int_to_lab":map_int_to_lab}, f"{savedir_preprocess}/map_int_to_lab-var_{var_decode}-var_others_{'|'.join(vars_others)}.txt")

                        # var_decode = "di_an_ci_ve_bin"
                        # var_decode = "velocity"
                        path_conj = f"{savedir_preprocess}/conjunctions-var_{var_decode}-var_others_{'|'.join(vars_others)}.png"
                        dflab_pruned, dict_dfthis = extract_with_levels_of_conjunction_vars(dflab, var_decode,
                                                                                vars_others,
                                                                                n_min_across_all_levs_var=n_min_across_all_levs_var,
                                                                                lenient_allow_data_if_has_n_levels=2,
                                                                                prune_levels_with_low_n=True,
                                                                                plot_counts_heatmap_savepath=path_conj)

                        if len(dflab_pruned)>0:

                            ### FIRST, a single decoder over entire data (after pruning to levels of othervar that have some data across var)
                            if len(dflab_pruned)==0:
                                print(len(dflab))
                                print(var_decode)
                                print(vars_others)
                                print("see conjunctions which failed at:", path_conj)
                                assert False

                            inds = dflab_pruned["_index"].tolist()
                            X = X_orig[inds, :]
                            labels = Labels_orig_int[inds]
                            plot_resampled_data_path_nosuff = f"{savedir_preprocess}/var-{var_decode}-ALL_DATA-{br}"
                            res = decode_categorical_wrapper(X, labels, n_min_across_all_levs_var, plot_resampled_data_path_nosuff)
                            plt.close("all")

                            # avearge over folds
                            score_mean = np.mean([r["score_xval"] for r in res])
                            score_std = np.std([r["score_xval"] for r in res])
                            score_adjusted_mean = np.mean([r["score_xval_adjusted"] for r in res])

                            RES_SINGLE.append({
                                "score_xval_mean":score_mean,
                                "score_adjusted_mean":score_adjusted_mean,
                                "score_xval_std":score_std,
                                "nclasses":len(set(labels)),
                                "n_dat":len(labels),
                                "n_splits":res[0]["n_splits"],
                                "n_min_across_labs":res[0]["n_min_across_labs"],
                                "n_max_across_labs":res[0]["n_max_across_labs"],
                                "bregion":br,
                                "var_decode":var_decode,
                                "vars_others":tuple(vars_others)
                            })
                            PLOT_RESAMPLED_DATA = False
                            # For each level of "others" do decoding

                            for levo, dfthis in dict_dfthis.items():

                                # Do decode/test
                                inds = dfthis["_index"].tolist()
                                X = X_orig[inds, :]
                                labels = Labels_orig_int[inds]

                                # Sanity check, same data across all bregions.
                                from pythonlib.tools.checktools import check_objects_identical
                                if levo in map_levother_to_labels.items():
                                    if not check_objects_identical(map_levother_to_labels[levo], labels):
                                        print(map_levother_to_labels[levo])
                                        print(labels)
                                        assert False
                                else:
                                    map_levother_to_labels[levo]=labels

                                plot_resampled_data_path_nosuff = f"{savedir_preprocess}/var-{var_decode}-lev_other_{levo}-{br}"
                                res = decode_categorical_wrapper(X, labels, n_min_across_all_levs_var, plot_resampled_data_path_nosuff)
                                plt.close("all")

                                # avearge over folds
                                score_mean = np.mean([r["score_xval"] for r in res])
                                score_std = np.std([r["score_xval"] for r in res])
                                score_adjusted_mean = np.mean([r["score_xval_adjusted"] for r in res])

                                RES.append({
                                    "lev_other":levo,
                                    "score_xval_mean":score_mean,
                                    "score_adjusted_mean":score_adjusted_mean,
                                    "score_xval_std":score_std,
                                    "nclasses":len(set(labels)),
                                    "n_dat":len(labels),
                                    "n_splits":res[0]["n_splits"],
                                    "n_min_across_labs":res[0]["n_min_across_labs"],
                                    "n_max_across_labs":res[0]["n_max_across_labs"],
                                    "bregion":br,
                                    "var_decode":var_decode,
                                    "vars_others":tuple(vars_others)
                                })
                    elif conti_or_discr=="conti":
                        # Continuous -- re-bin within each level of othervar...
                        PLOT_RESAMPLED_DATA = False
                        # For each level of "others" do decoding
                        n_min_across_all_levs_var = 8 # min n in each class

                        # For motor decoding, use bins local to the sahpe (i.e., recalcualte teh bins),
                        # and do separately for each feature.

                        Labels_orig = dflab[var_decode]
                        assert len(Labels_orig)==X_orig.shape[0]

                        # SKIP THis, since it resets the indices...
                        # from pythonlib.tools.pandastools import extract_with_levels_of_var_good
                        # dfthis, inds_keep = extract_with_levels_of_var_good(dflab, vars_others, n_min_across_all_levs_var*2)


                        ### FIRST, a single decoder over entire data (after pruning to levels of othervar that have some data across var)
                        X = X_orig
                        labels = Labels_orig

                        # Bin data.
                        from pythonlib.tools.nptools import bin_values, bin_values_by_rank
                        nbins = int(np.floor(len(labels)/n_min_across_all_levs_var))
                        if nbins>6:
                            nbins = 6
                        labels = bin_values_by_rank(labels, nbins=nbins)

                        plot_resampled_data_path_nosuff = f"{savedir_preprocess}/var-{var_decode}-ALL_DATA-{br}"
                        res = decode_categorical_wrapper(X, labels, n_min_across_all_levs_var, plot_resampled_data_path_nosuff)
                        plt.close("all")

                        # avearge over folds
                        score_mean = np.mean([r["score_xval"] for r in res])
                        score_std = np.std([r["score_xval"] for r in res])
                        score_adjusted_mean = np.mean([r["score_xval_adjusted"] for r in res])

                        RES_SINGLE.append({
                            "score_xval_mean":score_mean,
                            "score_adjusted_mean":score_adjusted_mean,
                            "score_xval_std":score_std,
                            "nclasses":len(set(labels)),
                            "n_dat":len(labels),
                            "n_splits":res[0]["n_splits"],
                            "n_min_across_labs":res[0]["n_min_across_labs"],
                            "n_max_across_labs":res[0]["n_max_across_labs"],
                            "bregion":br,
                            "var_decode":var_decode,
                            "vars_others":tuple(vars_others)
                        })

                        from pythonlib.tools.pandastools import grouping_append_and_return_inner_items
                        groupdict = grouping_append_and_return_inner_items(dflab, vars_others)


                        for levo, inds in groupdict.items():

                            if len(inds)>=n_min_across_all_levs_var*2:
                                # Otherwise cant get >1 class for decoding

                                # Do decode/test
                                X = X_orig[inds, :]
                                labels = Labels_orig[inds]

                                # Bin data.
                                from pythonlib.tools.nptools import bin_values, bin_values_by_rank
                                nbins = int(np.floor(len(labels)/n_min_across_all_levs_var))
                                if nbins>6:
                                    nbins = 6
                                labels = bin_values_by_rank(labels, nbins=nbins)

                                # Sanity check, same data across all bregions.
                                from pythonlib.tools.checktools import check_objects_identical
                                if levo in map_levother_to_labels.items():
                                    if not check_objects_identical(map_levother_to_labels[levo], labels):
                                        print(map_levother_to_labels[levo])
                                        print(labels)
                                        assert False
                                else:
                                    map_levother_to_labels[levo]=labels

                                from neuralmonkey.analyses.decode_good import decode_categorical_wrapper
                                plot_resampled_data_path_nosuff = f"{savedir_preprocess}/var-{var_decode}-lev_other_{levo}-{br}"
                                res = decode_categorical_wrapper(X, labels, n_min_across_all_levs_var, plot_resampled_data_path_nosuff)
                                plt.close("all")

                                # avearge over folds
                                score_mean = np.mean([r["score_xval"] for r in res])
                                score_std = np.std([r["score_xval"] for r in res])
                                score_adjusted_mean = np.mean([r["score_xval_adjusted"] for r in res])

                                RES.append({
                                    "lev_other":levo,
                                    "score_xval_mean":score_mean,
                                    "score_adjusted_mean":score_adjusted_mean,
                                    "score_xval_std":score_std,
                                    "nclasses":len(set(labels)),
                                    "n_dat":len(labels),
                                    "n_splits":res[0]["n_splits"],
                                    "n_min_across_labs":res[0]["n_min_across_labs"],
                                    "n_max_across_labs":res[0]["n_max_across_labs"],
                                    "bregion":br,
                                    "var_decode":var_decode,
                                    "vars_others":tuple(vars_others)
                                })
                    else:
                        print(conti_or_discr)
                        assert False


            ##### SAVE DATA
            savedir = f"{SAVEDIR}/summary_plots"
            os.makedirs(savedir, exist_ok=True)
            dfres_single = pd.DataFrame(RES_SINGLE)
            dfres = pd.DataFrame(RES)

            dfres_single.to_csv(f"{SAVEDIR}/dfres_single.csv")
            dfres.to_csv(f"{SAVEDIR}/dfres.csv")

            pd.to_pickle(dfres_single, f"{SAVEDIR}/dfres_single.pkl")
            pd.to_pickle(dfres, f"{SAVEDIR}/dfres.pkl")

            ####### PLOTS
            plot_summary_decode_results(dfres, dfres_single, SAVEDIR)

            #
            # from pythonlib.tools.snstools import rotateLabel
            # import seaborn as sns
            # fig = sns.catplot(data=dfres_single, x="bregion", y="score_xval_mean", hue="nclasses", row="var_decode", col="vars_others")
            # plt.axhline(0)
            # plt.axhline(1)
            # rotateLabel(fig)
            # savefig(fig, f"{savedir}/single_decoder_across_all_othervar-1.pdf")
            #
            # fig = sns.catplot(data=dfres_single, x="bregion", y="score_xval_mean", hue="nclasses", row="var_decode", col="vars_others", kind="bar", ci=68)
            # plt.axhline(0)
            # plt.axhline(1)
            # rotateLabel(fig)
            # savefig(fig, f"{savedir}/single_decoder_across_all_othervar-2.pdf")
            #
            # fig = sns.catplot(data=dfres_single, x="bregion", y="score_adjusted_mean", row="var_decode", col="vars_others", kind="bar", ci=68)
            # plt.axhline(0)
            # plt.axhline(1)
            # rotateLabel(fig)
            # savefig(fig, f"{savedir}/single_decoder_across_all_othervar-score_adjusted-1.pdf")
            #
            # ################################
            # import seaborn as sns
            # fig = sns.catplot(data=dfres, x="bregion", y="score_xval_mean", hue="nclasses", row="var_decode", col="vars_others")
            # plt.axhline(0)
            # plt.axhline(1)
            # rotateLabel(fig)
            # savefig(fig, f"{savedir}/separate_decoder_each_othervar-1.pdf")
            #
            # fig = sns.catplot(data=dfres, x="bregion", y="score_xval_mean", hue="nclasses", row="var_decode", col="vars_others", kind="bar", ci=68)
            # plt.axhline(0)
            # plt.axhline(1)
            # rotateLabel(fig)
            # savefig(fig, f"{savedir}/separate_decoder_each_othervar-2.pdf")
            #
            # fig = sns.catplot(data=dfres, x="bregion", y="score_adjusted_mean", row="var_decode", col="vars_others", kind="bar", ci=68)
            # plt.axhline(0)
            # plt.axhline(1)
            # rotateLabel(fig)
            # savefig(fig, f"{savedir}/separate_decoder_each_othervar-score_adjusted-1.pdf")
            #
            # plt.close("all")


[(-0.3, 0)]
Searching using this string:
/mnt/Freiwald/ltian/recordings/*Pancho*/*220719*/**
Found this many paths:
1
---
/mnt/Freiwald/ltian/recordings/Pancho/220719/Pancho-220719-155517
session:  0
------------------------------
Loading this neural session: 0
Loading these beh expts: ['priminvar3e', 'priminvar3e']
Loading these beh sessions: [1, 2]
Using this beh_trial_map_list: [(1, 0), (1, 45)]
Searching using this string:
/mnt/Freiwald/ltian/recordings/*Pancho*/*220719*/**
Found this many paths:
1
---
/mnt/Freiwald/ltian/recordings/Pancho/220719/Pancho-220719-155517
{'filename_components_hyphened': ['Pancho', '220719', '155517'], 'basedirs': ['/mnt/Freiwald/ltian/recordings/Pancho', '/mnt/Freiwald/ltian/recordings/Pancho/220719'], 'basedirs_filenames': ['220719', 'Pancho-220719-155517'], 'filename_final_ext': 'Pancho-220719-155517', 'filename_final_noext': 'Pancho-220719-155517'}
FOund this path for spikes:  /mnt/Freiwald/ltian/recordings/Pancho/220719/Pancho-220719-155517/spikes_

/gorilla1/code/pythonlib/pythonlib/drawmodel/features.py:183: RuntimeWarning: invalid value encountered in double_scalars
  return [1-p/t for p,t in zip(displace,distance)]


Num nan/total, for circ
1 / 672
Num nan/total, for dist
0 / 672
Added these features:
['FEAT_angle_overall', 'FEAT_num_strokes_beh', 'FEAT_num_strokes_task', 'FEAT_circ', 'FEAT_dist']
.. Appended new column 'char_seq', version: task_matlab
Starting length of D.Dat: 672
--BEFORE REMOVE; existing supervision_stage_concise:
off|0||1111|0    672
Name: supervision_stage_concise, dtype: int64
############ TAKING ONLY NO SUPERVISION TRIALS
*** RUNNING D.preprocessGood using these params:
['no_supervision']
-- Len of D, before applying this param: no_supervision, ... 672
after: 672
Dataset final len: 672
*** RUNNING D.preprocessGood using these params:
['remove_online_abort', 'beh_strokes_at_least_one']
-- Len of D, before applying this param: remove_online_abort, ... 672
after: 671
-- Len of D, before applying this param: beh_strokes_at_least_one, ... 671
after: 671
clean_preprocess_data...
len of DS.Dat = 1397, before running... dataset_missing_stroke_in_context
Removing these inds from DS, 

/home/lucast4/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/dPCA/dPCA.py:681: RuntimeWarning: divide by zero encountered in true_divide
  trainX = (X*(N_samples/(N_samples-1))[(np.s_[:],)*n_unprotect + (None,)*n_protect] - blindX/(N_samples-1)[(np.s_[:],)*n_unprotect + (None,)*n_protect])
/home/lucast4/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/dPCA/dPCA.py:681: RuntimeWarning: invalid value encountered in true_divide
  trainX = (X*(N_samples/(N_samples-1))[(np.s_[:],)*n_unprotect + (None,)*n_protect] - blindX/(N_samples-1)[(np.s_[:],)*n_unprotect + (None,)*n_protect])
/home/lucast4/miniconda3/envs/drag2_matlab/lib/python3.8/site-packages/dPCA/dPCA.py:681: RuntimeWarning: invalid value encountered in subtract
  trainX = (X*(N_samples/(N_samples-1))[(np.s_[:],)*n_unprotect + (None,)*n_protect] - blindX/(N_samples-1)[(np.s_[:],)*n_unprotect + (None,)*n_protect])


ValueError: array must not contain infs or NaNs

In [39]:
dpca_marginalization

'shape'

In [41]:
# Clean up PA
print(pa.X.shape)
pa_out, res_check_tasksets, res_check_effectvars = preprocess_rsa_prepare_popanal_wrapper(pa, [dpca_marginalization], False, False, None, None, False, None)
print(pa_out.X.shape)



(57, 540, 10)
[CHECK CONJ] NOTHING TO REMOVE!
[CHECK CONJ] NOTHING TO REMOVE!
(57, 532, 10)


In [42]:
dpca, Z, R, trialR, map_var_to_lev, map_grp_to_idx, params_dpca, panorm = dpca_compute_pa_to_space(
    pa_out, [dpca_marginalization], keep_all_margs=False)


(30, 57, 22, 10)
('Lcentered-6-3-0',) [12, 25, 57, 87, 89, 115, 163, 175, 211, 222, 240, 246, 258, 267, 288, 339, 357, 375, 392, 400, 403, 424, 456, 499, 504, 513, 530]
dict_keys([('Lcentered-6-3-0',), ('Lcentered-6-4-0',), ('Lcentered-6-5-0',), ('Lcentered-6-6-0',), ('Lcentered-6-8-0',), ('Lzigzag1-4-5-0',), ('Lzigzag1-4-5-1',), ('Lzigzag1-4-6-0',), ('V-2-1-0',), ('V-2-2-0',), ('V-2-4-0',), ('arcdeep-4-1-0',), ('arcdeep-4-2-0',), ('arcdeep-4-4-0',), ('circle-6-1-0',), ('line-9-1-0',), ('line-9-2-0',), ('line-9-3-0',), ('line-9-4-0',), ('squiggle3-3-1-0',), ('squiggle3-3-2-0',), ('squiggle3-3-2-1',)])
('Lcentered-6-4-0',) [20, 52, 54, 69, 98, 122, 152, 160, 231, 254, 322, 371, 426, 489, 512]
dict_keys([('Lcentered-6-3-0',), ('Lcentered-6-4-0',), ('Lcentered-6-5-0',), ('Lcentered-6-6-0',), ('Lcentered-6-8-0',), ('Lzigzag1-4-5-0',), ('Lzigzag1-4-5-1',), ('Lzigzag1-4-6-0',), ('V-2-1-0',), ('V-2-2-0',), ('V-2-4-0',), ('arcdeep-4-1-0',), ('arcdeep-4-2-0',), ('arcdeep-4-4-0',), ('circle-6-1-

# Load saved decoding results

In [ ]:
from neuralmonkey.scripts.analyquick_decode_substrokes import load_saved_data, plot_summary_decode_results
import os
import matplotlib.pyplot as plt
from pythonlib.tools.plottools import savefig
from pythonlib.tools.snstools import rotateLabel
# import pandas as pd

In [ ]:

for animal, date in [
    # ("Diego", 230615),
    # ("Diego", 230616),
    # ("Diego", 230618),
    # ("Diego", 230619),
    # ("Pancho", 220715),
    # ("Pancho", 220719)
    # ]:   
    ("Pancho", 221217)
    ]:   
    # animal = "Diego"  
    # date = 230615 
    
    for prune_to_first_substroke in [False, True]:
        for twind in [(-0.3, 0), (0.0, 0.3)]:
            # prune_to_first_substroke = False 
            # twind = (-0.3, 0)
            try:
                dfres, dfres_single, SAVEDIR = load_saved_data(animal, date, prune_to_first_substroke, twind)
                print("Loaded data from: ", SAVEDIR)
                plot_summary_decode_results(dfres, dfres_single, SAVEDIR)
            except Exception as err:
                print("DID NOT FIND: ", animal, date, prune_to_first_substroke, twind)
                pass
        